In [3]:
!pip install rapidfuzz yfinance pandas numpy

  Using cached rapidfuzz-3.14.1-cp311-cp311-win_amd64.whl.metadata (12 kB)
  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached frozendict-2.4.6-py311-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.2-py3-none-any.whl
  Using cached curl_cffi-0.13.0-cp39-abi3-win_amd64.whl.metadata (13 kB)
  Using cached websockets-15.0.1-cp311-cp311-win_amd64.whl.metadata (7.0 kB)
Using cached rapidfuzz-3.14.1-cp311-cp311-win_amd64.whl (1.5 MB)
Using cached yfinance-0.2.66-py2.py3-none-any.whl (123 kB)
Using cached curl_cffi-0.13.0-cp39-abi3-win_amd64.whl (1.6 MB)
Using cached frozendict-2.4.6-py311-none-any.whl (16 kB)
   ---------------------------------------- 0.0/436.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/436.9 kB ? eta -:--:--
    --------------------------------------- 10.2/436.9 kB ? eta -:--:--
   - ------------------------------------- 20.5/436.9 kB 217.9 kB/s eta 0:00:02
 


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: C:\Users\Crook\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
from rapidfuzz import process, fuzz
import yfinance as yf
import pandas as pd
import numpy as np

import json

In [42]:
# LOAD SEC(https://www.sec.gov/files/company_tickers.json) TICKERS
with open("tickers.json", "r") as f:
    tickers = json.load(f)
    companies = [tickers[i]["title"] for i in tickers]
    abbrs = [tickers[i]["ticker"] for i in tickers]

    comp_tckr = {comp: abbr for comp, abbr in zip(companies, abbrs)}

# LOAD INITIAL RECALL DATA
df = pd.read_csv("recalls.csv")

In [27]:
#df

## Data Cleaning

In [43]:
# delete instances where year is older than 1995
print(len(df))
for i, row in df.iterrows():
    if ( row['Report Received Date'][-4:].isnumeric() and int(row['Report Received Date'][-4:]) < 1995 ):
        df.drop(i, inplace=True)
print(len(df))

29374
22505


In [ ]:
# remove column that won't feed into training
df = df.drop("Completion Rate % (Blank - Not Reported)", axis=1)

In [45]:
df

,Report Received Date,NHTSA ID,Recall Link,Manufacturer,Subject,Component,Mfr Campaign Number,Recall Type,Potentially Affected,Recall Description,Consequence Summary,Corrective Action,Park Outside Advisory,Do Not Drive Advisory,Stock Abbreviation,Opening Stock Value,Closing Stock Value
0,10/01/2025,25V656000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Toyota Motor Engineering & Manufacturing,Driveshaft May Deform and Break,POWER TRAIN,25TB11 / 25TA11,Vehicle,5960,Toyota Motor Engineering & Manufacturing (Toyo...,A broken driveshaft can impair steering. Vehic...,Dealers will inspect and replace both front dr...,No,No,NYSE:TM,192.39,191.83
1,10/01/2025,25V655000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Nissan North America, Inc.",Fire Risk from Quick Charging Battery,ELECTRICAL SYSTEM,R25C8,Vehicle,19077,"Nissan North America, Inc. (Nissan) is recalli...",A quick charging battery that overheats increa...,Owners are advised not to use Level 3 quick ch...,No,No,NYSE:NSANY,NaN,NaN
2,09/30/2025,25V654000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Volvo Car USA, LLC",Power Operated Tailgate May Drop Suddenly,STRUCTURE,R10342,Vehicle,1119,"Volvo Car USA, LLC (Volvo) is recalling certai...",A tailgate that suddenly drops can hit a perso...,Owners are advised not to use the POT function...,No,No,NYSE:,NaN,NaN
3,09/30/2025,25V653000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Thor Motor Coach,LP Tank May Detach and Become a Road Hazard,EQUIPMENT,RC000331,Vehicle,23,Thor Motor Coach (TMC) is recalling certain 20...,A detached tank can create a road hazard for o...,The remedy is currently under development. Own...,No,No,NYSE:,NaN,NaN
4,09/26/2025,25E062000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"MEDIX SPECIALTY VEHICLES, INC.",Quick Liner Wheel Covers May Detach and Become...,WHEELS,NaN,Equipment,396,"Medix Specialty Vehicles, Inc. (Medix) is reca...",A detached cover can create a road hazard for ...,"Medix will replace the covers, free of charge....",No,No,NYSE:,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23948,01/11/1995,95V004000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"THOMAS BUILT BUSES, INC.",INTERIOR SYSTEMS:ACTIVE RESTRAINTS:BELT RETRAC...,SEAT BELTS,NR (Not Reported),Vehicle,3076,THE METAL HOUSING OF THE SAFETY BELT RETRACTOR...,PASSENGERS WOULD BE UNABLE TO EXTEND OR RETRAC...,DEALERS WILL INSTALL A HARDENED WASHER WHICH W...,No,No,NYSE:,NaN,NaN
23949,01/03/1995,95T001000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Cooper Tire & Rubber Co.,TIRES:MARKINGS,TIRES,NR (Not Reported),Tire,6603,THE MAXIMUM INFLATION PRESSURE STAMPING ON THE...,NaN,DEALERS WILL REPLACE THE INCORRECT TIRES WITH ...,No,No,NYSE:,NaN,NaN
23950,01/03/1995,95V013000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"PREVOST CAR, INC. Inactive",STRUCTURE:DOOR ASSEMBLY,STRUCTURE,NR (Not Reported),Vehicle,1090,IF A PASSENGER TRIES TO EXIT THE LAVATORY COMP...,"IF THE WINDOW SEPARATES, THE OCCUPANT CAN FALL...",DEALERS WILL INSTALL A RAIL IN THE LAVATORY CO...,No,No,NYSE:,NaN,NaN
23951,01/03/1995,95V016000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"PREVOST CAR, INC. Inactive",EMERGENCY PARKING BRAKE:MECHANICAL,PARKING BRAKE,NR (Not Reported),Vehicle,2075,THE PARKING BRAKE BUTTON CAN BE HIT ACCIDENTLY...,UNINTENDED OPERATION OF THE PARKING BRAKE BUTT...,DEALERS WILL INSTALL A PROTECTOR OVER THE PARK...,No,No,NYSE:,NaN,NaN


In [54]:
import re
from collections import Counter

# Common words to ignore
stop_words = set([
    "inc", "corp", "llc", "co", "ltd", "corporation",
    "company", "limited", "plc", "gmbh", "sa", "ag", "pte", "bv", "kg", "kgaa", "corp", "industries", "us", "of"
])

def tokenize(name):
    name = name.lower()
    name = re.sub(r'[^a-z0-9 ]', ' ', name)  # remove punctuation
    words = name.split()
    return [w for w in words if w not in stop_words]

# Build a word-to-company mapping for fast lookup
word_counts = Counter()
company_word_map = {}
for c in companies:
    words = tokenize(c)
    company_word_map[c] = set(words)
    word_counts.update(words)


def match_by_unique_words(name):
    words = tokenize(name)
    if not words:
        return None
    
    # Score companies by sum of inverse word frequencies (rare words are more valuable)
    scores = {}
    for c, c_words in company_word_map.items():
        score = sum(1 / word_counts[w] for w in words if w in c_words)
        if score > 0:
            scores[c] = score
    
    if scores:
        return max(scores, key=scores.get)
    return None

df.loc[:2000, "fuzzy_company"] = df.loc[:2000, "Manufacturer"].apply(match_by_unique_words)

In [55]:
df

,Report Received Date,NHTSA ID,Recall Link,Manufacturer,Subject,Component,Mfr Campaign Number,Recall Type,Potentially Affected,Recall Description,...,Opening Stock Value,Closing Stock Value,fuzzy_company,ticker,open,close,market_cap,eps,full_name,market_cap_on_date
0,2025-10-01,25V656000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Toyota Motor Engineering & Manufacturing,Driveshaft May Deform and Break,POWER TRAIN,25TB11 / 25TA11,Vehicle,5960,Toyota Motor Engineering & Manufacturing (Toyo...,...,192.39,191.83,TOYOTA MOTOR CORP/,TOYOF,17.840000,19.379999,266315628544,2.16,Toyota Motor Corporation,19.379999
1,2025-10-01,25V655000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Nissan North America, Inc.",Fire Risk from Quick Charging Battery,ELECTRICAL SYSTEM,R25C8,Vehicle,19077,"Nissan North America, Inc. (Nissan) is recalli...",...,NaN,NaN,Central North Airport Group,GAERF,13.940000,13.940000,4889556480,0.73,"Grupo Aeroportuario del Centro Norte, S.A.B. d...",13.940000
2,2025-09-30,25V654000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Volvo Car USA, LLC",Power Operated Tailgate May Drop Suddenly,STRUCTURE,R10342,Vehicle,1119,"Volvo Car USA, LLC (Volvo) is recalling certai...",...,NaN,NaN,"Mister Car Wash, Inc.",MCW,5.300000,5.330000,1685467520,0.26,"Mister Car Wash, Inc.",5.330000
3,2025-09-30,25V653000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Thor Motor Coach,LP Tank May Detach and Become a Road Hazard,EQUIPMENT,RC000331,Vehicle,23,Thor Motor Coach (TMC) is recalling certain 20...,...,NaN,NaN,THOR INDUSTRIES INC,THO,105.050003,103.690002,5738598400,4.84,"THOR Industries, Inc.",103.690002
4,2025-09-26,25E062000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"MEDIX SPECIALTY VEHICLES, INC.",Quick Liner Wheel Covers May Detach and Become...,WHEELS,NaN,Equipment,396,"Medix Specialty Vehicles, Inc. (Medix) is reca...",...,NaN,NaN,"Envirotech Vehicles, Inc.",EVTV,2.020000,1.890000,5472346,-10.19,"Envirotech Vehicles, Inc.",1.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2025-07-25,25V489000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Seat Belt Warning System Malfunction/FMVSS 208,SEAT BELTS,25C31,Vehicle,56473,Ford Motor Company (Ford) is recalling certain...,...,11.34,11.47,FORD MOTOR CO,F-PD,NaN,NaN,None,None,None,NaN
185,2025-07-25,25V488000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Loss of Power Brake Assist,"SERVICE BRAKES, HYDRAULIC",25S77,Vehicle,312120,Ford Motor Company (Ford) is recalling certain...,...,11.34,11.47,FORD MOTOR CO,F-PD,NaN,NaN,None,None,None,NaN
186,2025-07-25,25V487000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Mercedes-Benz USA, LLC",Loss of Drive Power,ELECTRICAL SYSTEM,NaN,Vehicle,660,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",...,NaN,NaN,COMFORT SYSTEMS USA INC,FIX,664.826202,688.249512,29107197952,19.45,"Comfort Systems USA, Inc.",688.249512
187,2025-07-25,25V486000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Forest River, Inc.",Fuel Line Extension May Detach and Cause Fire,"FUEL SYSTEM, GASOLINE",51-1955,Vehicle,534,"Forest River, Inc. (Forest River) is recalling...",...,NaN,NaN,"CHARLES RIVER LABORATORIES INTERNATIONAL, INC.",CRL,177.149994,176.929993,9398431744,-1.31,"Charles River Laboratories International, Inc.",176.929993


In [56]:
# GET STOCK TICKER
df["ticker"] = df["fuzzy_company"].map(comp_tckr)

# ENSURE DATETIME
df["Report Received Date"] = pd.to_datetime(df["Report Received Date"])

# GET ALL PRICING DATA
unique_tickers = df["ticker"].astype(str).unique().tolist()
start_date = df["Report Received Date"].min()
end_date = df["Report Received Date"].max() + pd.Timedelta(days=1)
price_data = yf.download(unique_tickers, start=start_date, end=end_date, group_by="ticker", progress=False)

# HELPER TO GET PRICE DATA
def get_price(ticker, date):
    try:
        day_data = price_data[ticker].loc[date.strftime("%Y-%m-%d")]
        return pd.Series([day_data["Open"], day_data["Close"]])
    except Exception:
        return pd.Series([None, None])

# OBTAIN OPENING AND CLOSING PRICE
df[["open", "close"]] = df.apply(lambda x: get_price(x["ticker"], x["Report Received Date"]), axis=1)

# GRAB CURRENT INFO
info_cache = {}
for ticker in unique_tickers:
    try:
        t = yf.Ticker(ticker)
        info = t.info
        info_cache[ticker] = {
            "market_cap": info.get("marketCap"),
            "eps": info.get("trailingEps"),
            "full_name": info.get("longName"),
        }
    except Exception:
        pass

# CREATE AND MERGE INFO
info_df = pd.DataFrame(info_cache).T
info_df.index.name = "ticker"
df = df.merge(info_df, on="ticker", how="left")

# ADJUST CAP
df["market_cap_on_date"] = df["close"] #* df["shares_outstanding"]

C:\Users\Crook\AppData\Local\Temp\ipykernel_41968\753775025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["ticker"] = df["fuzzy_company"].map(comp_tckr)
C:\Users\Crook\AppData\Local\Temp\ipykernel_41968\753775025.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Report Received Date"] = pd.to_datetime(df["Report Received Date"])
C:\Users\Crook\AppData\Local\Temp\ipykernel_41968\753775025.py:11: FutureWarning: YF.download() has changed argument auto_adjust default to True
  price_data = yf.dow

In [57]:
df

,Report Received Date,NHTSA ID,Recall Link,Manufacturer,Subject,Component,Mfr Campaign Number,Recall Type,Potentially Affected,Recall Description,...,ticker,open,close,market_cap_x,eps_x,full_name_x,market_cap_on_date,market_cap_y,eps_y,full_name_y
0,2025-10-01,25V656000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Toyota Motor Engineering & Manufacturing,Driveshaft May Deform and Break,POWER TRAIN,25TB11 / 25TA11,Vehicle,5960,Toyota Motor Engineering & Manufacturing (Toyo...,...,TOYOF,17.840000,19.379999,266315628544,2.16,Toyota Motor Corporation,19.379999,266315628544,2.16,Toyota Motor Corporation
1,2025-10-01,25V655000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Nissan North America, Inc.",Fire Risk from Quick Charging Battery,ELECTRICAL SYSTEM,R25C8,Vehicle,19077,"Nissan North America, Inc. (Nissan) is recalli...",...,GAERF,13.940000,13.940000,4889556480,0.73,"Grupo Aeroportuario del Centro Norte, S.A.B. d...",13.940000,4889556480,0.73,"Grupo Aeroportuario del Centro Norte, S.A.B. d..."
2,2025-09-30,25V654000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Volvo Car USA, LLC",Power Operated Tailgate May Drop Suddenly,STRUCTURE,R10342,Vehicle,1119,"Volvo Car USA, LLC (Volvo) is recalling certai...",...,MCW,5.300000,5.330000,1685467520,0.26,"Mister Car Wash, Inc.",5.330000,1685467520,0.26,"Mister Car Wash, Inc."
3,2025-09-30,25V653000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Thor Motor Coach,LP Tank May Detach and Become a Road Hazard,EQUIPMENT,RC000331,Vehicle,23,Thor Motor Coach (TMC) is recalling certain 20...,...,THO,105.050003,103.690002,5738598400,4.84,"THOR Industries, Inc.",103.690002,5738598400,4.84,"THOR Industries, Inc."
4,2025-09-26,25E062000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"MEDIX SPECIALTY VEHICLES, INC.",Quick Liner Wheel Covers May Detach and Become...,WHEELS,NaN,Equipment,396,"Medix Specialty Vehicles, Inc. (Medix) is reca...",...,EVTV,2.020000,1.890000,5472346,-10.19,"Envirotech Vehicles, Inc.",1.890000,5472346,-10.19,"Envirotech Vehicles, Inc."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
143,2025-07-25,25V489000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Seat Belt Warning System Malfunction/FMVSS 208,SEAT BELTS,25C31,Vehicle,56473,Ford Motor Company (Ford) is recalling certain...,...,F-PD,NaN,NaN,None,None,None,NaN,None,None,None
144,2025-07-25,25V488000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Loss of Power Brake Assist,"SERVICE BRAKES, HYDRAULIC",25S77,Vehicle,312120,Ford Motor Company (Ford) is recalling certain...,...,F-PD,NaN,NaN,None,None,None,NaN,None,None,None
145,2025-07-25,25V487000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Mercedes-Benz USA, LLC",Loss of Drive Power,ELECTRICAL SYSTEM,NaN,Vehicle,660,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",...,FIX,664.826202,688.249512,29107197952,19.45,"Comfort Systems USA, Inc.",688.249512,29107197952,19.45,"Comfort Systems USA, Inc."
146,2025-07-25,25V486000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Forest River, Inc.",Fuel Line Extension May Detach and Cause Fire,"FUEL SYSTEM, GASOLINE",51-1955,Vehicle,534,"Forest River, Inc. (Forest River) is recalling...",...,CRL,177.149994,176.929993,9398431744,-1.31,"Charles River Laboratories International, Inc.",176.929993,9398431744,-1.31,"Charles River Laboratories International, Inc."


In [50]:
print(len(df))

22505


In [51]:
df = df[df['ticker'].notna()]

In [52]:
print(len(df))

148


In [53]:
df

,Report Received Date,NHTSA ID,Recall Link,Manufacturer,Subject,Component,Mfr Campaign Number,Recall Type,Potentially Affected,Recall Description,...,Opening Stock Value,Closing Stock Value,fuzzy_company,ticker,open,close,market_cap,eps,full_name,market_cap_on_date
0,2025-10-01,25V656000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Toyota Motor Engineering & Manufacturing,Driveshaft May Deform and Break,POWER TRAIN,25TB11 / 25TA11,Vehicle,5960,Toyota Motor Engineering & Manufacturing (Toyo...,...,192.39,191.83,TOYOTA MOTOR CORP/,TOYOF,17.840000,19.379999,266315628544,2.16,Toyota Motor Corporation,19.379999
1,2025-10-01,25V655000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Nissan North America, Inc.",Fire Risk from Quick Charging Battery,ELECTRICAL SYSTEM,R25C8,Vehicle,19077,"Nissan North America, Inc. (Nissan) is recalli...",...,NaN,NaN,Central North Airport Group,GAERF,13.940000,13.940000,4889556480,0.73,"Grupo Aeroportuario del Centro Norte, S.A.B. d...",13.940000
2,2025-09-30,25V654000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Volvo Car USA, LLC",Power Operated Tailgate May Drop Suddenly,STRUCTURE,R10342,Vehicle,1119,"Volvo Car USA, LLC (Volvo) is recalling certai...",...,NaN,NaN,"Mister Car Wash, Inc.",MCW,5.300000,5.330000,1685467520,0.26,"Mister Car Wash, Inc.",5.330000
3,2025-09-30,25V653000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Thor Motor Coach,LP Tank May Detach and Become a Road Hazard,EQUIPMENT,RC000331,Vehicle,23,Thor Motor Coach (TMC) is recalling certain 20...,...,NaN,NaN,THOR INDUSTRIES INC,THO,105.050003,103.690002,5738598400,4.84,"THOR Industries, Inc.",103.690002
4,2025-09-26,25E062000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"MEDIX SPECIALTY VEHICLES, INC.",Quick Liner Wheel Covers May Detach and Become...,WHEELS,NaN,Equipment,396,"Medix Specialty Vehicles, Inc. (Medix) is reca...",...,NaN,NaN,"Envirotech Vehicles, Inc.",EVTV,2.020000,1.890000,5472346,-10.19,"Envirotech Vehicles, Inc.",1.890000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2025-07-25,25V489000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Seat Belt Warning System Malfunction/FMVSS 208,SEAT BELTS,25C31,Vehicle,56473,Ford Motor Company (Ford) is recalling certain...,...,11.34,11.47,FORD MOTOR CO,F-PD,NaN,NaN,None,None,None,NaN
185,2025-07-25,25V488000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,Ford Motor Company,Loss of Power Brake Assist,"SERVICE BRAKES, HYDRAULIC",25S77,Vehicle,312120,Ford Motor Company (Ford) is recalling certain...,...,11.34,11.47,FORD MOTOR CO,F-PD,NaN,NaN,None,None,None,NaN
186,2025-07-25,25V487000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Mercedes-Benz USA, LLC",Loss of Drive Power,ELECTRICAL SYSTEM,NaN,Vehicle,660,"Mercedes-Benz USA, LLC (MBUSA) is recalling ce...",...,NaN,NaN,COMFORT SYSTEMS USA INC,FIX,664.826202,688.249512,29107197952,19.45,"Comfort Systems USA, Inc.",688.249512
187,2025-07-25,25V486000,Go to Recall (https://www.nhtsa.gov/recalls?nh...,"Forest River, Inc.",Fuel Line Extension May Detach and Cause Fire,"FUEL SYSTEM, GASOLINE",51-1955,Vehicle,534,"Forest River, Inc. (Forest River) is recalling...",...,NaN,NaN,"CHARLES RIVER LABORATORIES INTERNATIONAL, INC.",CRL,177.149994,176.929993,9398431744,-1.31,"Charles River Laboratories International, Inc.",176.929993
